In [1]:
from google.colab import drive
import pandas as pd
import numpy as np

drive.mount('/content/drive')

# Load data
df = pd.read_csv("/content/drive/MyDrive/Summarization.csv")
df.head()

Mounted at /content/drive


,Unnamed: 0,text,summary
0,0,លោក Pep Guardiola គ្រូបង្វឹក Man City បាននិយាយ...,Man City ឈ្នះជើងឯក Premier League រដូវកាល ២០២២...
1,1,លោក Luis Enrique គ្រូបង្វឹក Paris Saint Germai...,លោក Luis Enrique បដិសេធពាក្យចចាមអារ៉ាមថា ចង់ចា...
2,2,អង់គ្លេស៖ ក្រុមមាន់ខ្នាយមាស Tottenham Hotspur ...,Tottenham Hotspur នឹងប្រឹងប្រយុទ្ធប្រកួតជាមួយ ...
3,3,សហព័ន្ធបាល់ទាត់អឺរ៉ុប UEFA នឹងធ្វើសេចក្ដីសម្រេ...,UEFA នឹងសម្រេចក្នុងខែមិថុនាថា Barcelona ត្រូវដ...
4,4,ថ្មីៗនេះ មានរបាយការណ៍មកថា តារាឆ្នើមក្រុម Arsen...,Bukayo Saka កីឡាកររបស់ Arsenal កំពុងទាក់ទាញចំណ...


In [2]:
# Drop the unname column
df = df.drop('Unnamed: 0', axis=1)
df.head()

,text,summary
0,លោក Pep Guardiola គ្រូបង្វឹក Man City បាននិយាយ...,Man City ឈ្នះជើងឯក Premier League រដូវកាល ២០២២...
1,លោក Luis Enrique គ្រូបង្វឹក Paris Saint Germai...,លោក Luis Enrique បដិសេធពាក្យចចាមអារ៉ាមថា ចង់ចា...
2,អង់គ្លេស៖ ក្រុមមាន់ខ្នាយមាស Tottenham Hotspur ...,Tottenham Hotspur នឹងប្រឹងប្រយុទ្ធប្រកួតជាមួយ ...
3,សហព័ន្ធបាល់ទាត់អឺរ៉ុប UEFA នឹងធ្វើសេចក្ដីសម្រេ...,UEFA នឹងសម្រេចក្នុងខែមិថុនាថា Barcelona ត្រូវដ...
4,ថ្មីៗនេះ មានរបាយការណ៍មកថា តារាឆ្នើមក្រុម Arsen...,Bukayo Saka កីឡាកររបស់ Arsenal កំពុងទាក់ទាញចំណ...


In [3]:
# 1. Install necessary libraries
!pip install transformers datasets rouge-score sentencepiece


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=6a2ed7a0dea8dd7f4754c1af37e9979f648f35bdc0755b1607f5b92368893430
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score


In [4]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.6 MB/s eta 0:00:00


In [5]:
import torch
from torch.utils.data import DataLoader
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from datasets import load_dataset
import evaluate
import pandas as pd
from tqdm import tqdm

In [6]:
# Convert to HuggingFace dataset
from datasets import Dataset
dataset = Dataset.from_pandas(df)

In [7]:
# Split dataset
split_dataset = dataset.train_test_split(test_size=0.1)
train_dataset = split_dataset['train']
test_dataset = split_dataset['test']

In [8]:
# 4. Load tokenizer and model
model_name = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

# Set the tokenizer to Khmer (you can try 'km_KH')
tokenizer.src_lang = "km_KH"
tokenizer.tgt_lang = "km_KH"

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [9]:
# 5. Tokenization function
max_input_length = 512
max_target_length = 128

def preprocess(batch):
    inputs = tokenizer(batch['text'], max_length=max_input_length, truncation=True, padding='max_length', return_tensors="pt")
    targets = tokenizer(batch['summary'], max_length=max_target_length, truncation=True, padding='max_length', return_tensors="pt")
    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["labels"] = targets.input_ids
    return batch

train_dataset = train_dataset.map(preprocess, batched=True)
test_dataset = test_dataset.map(preprocess, batched=True)


Map:   0%|          | 0/2718 [00:00<?, ? examples/s]

Map:   0%|          | 0/303 [00:00<?, ? examples/s]

In [10]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    input_ids = [torch.tensor(item['input_ids']) for item in batch]
    attention_mask = [torch.tensor(item['attention_mask']) for item in batch]
    labels = [torch.tensor(item['labels']) for item in batch]

    # Pad sequences to the max length in the batch
    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask = pad_sequence(attention_mask, batch_first=True, padding_value=0)
    labels = pad_sequence(labels, batch_first=True, padding_value=-100)  # -100 so loss ignores padding

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': labels
    }

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=4, collate_fn=collate_fn)


In [11]:
# 7. Training setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)
num_epochs = 10

In [ ]:
import matplotlib.pyplot as plt

train_losses = []  # store average loss per epoch

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_loader):
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    train_losses.append(avg_loss)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")


100%|██████████| 680/680 [14:05<00:00,  1.24s/it]


Epoch 1/10, Loss: 1.7399


100%|██████████| 680/680 [14:16<00:00,  1.26s/it]


Epoch 2/10, Loss: 1.1639


100%|██████████| 680/680 [14:16<00:00,  1.26s/it]


...

100%|██████████| 680/680 [14:21<00:00,  1.27s/it]


Epoch 7/10, Loss: 0.3308


100%|██████████| 680/680 [14:21<00:00,  1.27s/it]


Epoch 8/10, Loss: 0.2443


 38%|███▊      | 256/680 [05:24<08:57,  1.27s/it]

In [ ]:
plt.figure()
plt.plot(range(1, num_epochs + 1), train_losses)
plt.xlabel("Epoch")
plt.ylabel("Training Loss")
plt.title("Training Loss vs Epoch")
plt.show()


In [ ]:
from evaluate import load_metric

# 9. Evaluation with ROUGE
rouge = load_metric("rouge")
model.eval()
predictions = []
references = []

for batch in tqdm(test_loader):
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    with torch.no_grad():
        generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=max_target_length)
    preds = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    refs = tokenizer.batch_decode(batch["labels"], skip_special_tokens=True)
    predictions.extend(preds)
    references.extend(refs)

results = rouge.compute(predictions=predictions, references=references)
print("ROUGE scores:", results)

In [ ]:
# 10. Example prediction
example_text = """
លោក Luis Enrique គ្រូបង្វឹក Paris Saint Germain បានចេញសេចក្តីថ្លែងការណ៍បដិសេធពាក្យចចាមអារ៉ាមថា រូបលោកកំពុងពិចារណាចាកចេញពីតំណែងនាពេលឆាប់ៗ។បុរសវ័យ៥៣ឆ្នាំ ត្រូវបានសារព័ត៌មានជាច្រើនផ្សព្វផ្សាយថា ចង់ចាកចេញពីក្លិបបារាំង ដោយសារតែបញ្ហាប្រឈម និងឥទ្ធិពលរបស់ Kylian Mbappe ប៉ុន្មានសប្តាហ៍ចុងក្រោយ បន្ទាប់ពីការតែងតាំងជាគ្រូបង្វឹកថ្មីរួចមក។Mbappe បច្ចុប្បន្នកំពុងនៅមានវិវាទជាមួយ PSG បន្ទាប់ពីបានបដិសេធបន្តកិច្ចសន្យាថ្មីនៅទឹកដី Parc Des Princes ដូច្នេះអាចធ្វើឱ្យគេចាកចេញដោយឥតគិតថ្លៃនៅចុងបញ្ចប់នៃរដូវកាលថ្មីនេះ។ ខ្សែប្រយុទ្ធបារាំងវ័យ២៤ឆ្នាំ ក៏ទើបតែបដិសេធសំណើដ៏ច្រើនសន្ធឹកសន្ធាប់ពីក្លិបនៅអារ៉ាប៊ីសាអូឌីត Al-Hilal ព្រមជាមួយនឹងការគ្មានឈ្មោះក្នុងដំណើរទេសចរណ៍មុនបើករដូវកាលរបស់ PSG ទៅកាន់ទ្វីបអាស៊ី ខណៈក្លិបទាមទារឱ្យរូបគេបន្តកិច្ចសន្យាថ្មី ឫជ្រើសរើសផ្លូវចាកចេញ។ជាមួយគ្នានេះដែរ លោក Luis Enrique ដែលទើបមកកាន់តំណែងនៅ Paris Saint Germain ជំនួសលោក Christophe Galtier កាលពីដើមខែកក្កដាកន្លងទៅ ត្រូវបានរាយការណ៍ថា មានបំណងចង់ចាកចេញតែម្តង ដោយសារធុញទ្រាន់នឹងរឿងរ៉ាវដែលកំពុងកើតឡើង។ ប៉ុន្តែ យ៉ាងណាក៏ដោយ ពេលនេះលោក Enrique បានធ្វើការបដិសេធការផ្សព្វផ្សាយទាំងអស់នោះ។លោក Enrique បានឱ្យដឹងថា៖ “ពាក្យចចាមអារ៉ាមទាំងនោះ គឺជារឿងគួរឱ្យអស់សំណើច ហើយវាស្ថិតនៅក្រៅការពិតទាំងស្រុង។ ខ្ញុំមិនមានបំណងចាកចេញទេ បើទោះជា Mbappe ចាកចេញក៏ដោយ។ ខ្ញុំនឹងចាកចេញពេលដែលស្ថានការណ៍មិនអាចទទួលយកបាន”។ក្លិប Paris Saint Germain បានសម្រេចនាំយកអតីតគ្រូបង្វឹក Barcelona និង ជម្រើសជាតិអេស្ប៉ាញ លោក  Enrique ដោយផ្តល់កុងត្រារហូតដល់ឆ្នាំ២០២៥ ក្នុងមហិច្ឆតា និងគោលបំណងឈ្នះពានរង្វាន់ UEFA Champions League ជាលើកដំបូងក្នុងប្រវត្តិសាស្រ្ត ក្រោយឈ្នះពាន Ligue ១ កាលពីរដូវកាលមុនជាលើកទី១១រួចមក។ទោះយ៉ាងណា២ប្រកួតចុងក្រោយ PSG ធ្វើមិនទាន់បានល្អទេក្នុងជំនួបមិត្តភាពនៅដើមខែសីហា ឆ្នាំ២០២៣ នាប្រទេសជប៉ុន ដោយផ្តួលក្រុមកូរ៉េ Jeonbuk Hyundai Motors ក្នុងលទ្ធផល ៣ទល់នឹង០ ក្រោយចាញ់ Inter Milan ក្នុងលទ្ធផល ២ ទល់នឹង ១៕,លោក Luis Enrique បដិសេធពាក្យចចាមអារ៉ាមថា ចង់ចាកចេញពី PSG ដោយសារបញ្ហារវាងក្លិប និង Mbappe ដែលបដិសេធបន្តកិច្ចសន្យាថ្មី។ សារព័ត៌មានរាយការណ៍ថា Enrique ធុញទ្រាន់នឹងស្ថានភាព ប៉ុន្តែគាត់បញ្ជាក់ថា ព័ត៌មានទាំងនោះគ្មានមូលដ្ឋានទេ ហើយនឹងនៅបន្តដឹកនាំ ទោះ Mbappe ចាកចេញក៏ដោយ។ Enrique ត្រូវបាន PSG តែងតាំងនៅខែកក្កដា ឆ្នាំ២០២៣ ជំនួស Galtier ដោយផ្តល់កុងត្រារហូតដល់ឆ្នាំ២០២៥ ដើម្បីនាំក្លិបឈ្នះ UEFA Champions League។ ទោះបីជាការប្រកួតមិត្តភាពចុងក្រោយមិនល្អក៏ដោយ គាត់និយាយថានឹងបន្តអភិវឌ្ឍក្រុម។ អង់គ្លេស៖ ក្រុមមាន់ខ្នាយមាស Tottenham Hotspur នឹងប្រឹងប្រែងប្រកួតប្រជែងដើម្បីស្វែងរកនូវការជំរុញក្តីសង្ឃឹមរបស់ពួកគេក្នុងការដណ្តើមបានចំណាត់ថ្នាក់កំពូលទាំងបួន (Top 4) នៅក្នុងក្របខណ្ឌ Premier League នៅពេលដែលពួកគេនឹងស្វាគមន៍ក្រុមបិសាចក្រហម Manchester United មកកាន់ North London នៅយប់ថ្ងៃអាទិត្យ ម៉ោង១០៖៣០នាទី (ម៉ោងនៅកម្ពុជា)។ បច្ចុប្បន្ន ក្រុមមាន់ខ្នាយមាស Tottenham Hotspur កំពុងឈរនៅចំណាត់ថ្នាក់លេខ៦ ក្នុងតារាងដោយមាន ៣ ពិន្ទុ តាមពីក្រោយក្រុមលេខ៤ West Ham United ខណៈក្រុមលេខ២ Man United មានសុវត្ថិភាពក្នុងការកក់កៅអីរបស់ខ្លួនសម្រាប់ការប្រកួត Champions League នៅឆ្នាំក្រោយ ទោះនៅសល់ ៨ ប្រកួតទៀតក៏ដោយ។
"""
inputs = tokenizer(example_text, return_tensors="pt", truncation=True, padding=True, max_length=max_input_length).to(device)
summary_ids = model.generate(inputs['input_ids'], max_length=max_target_length)
print("Generated Summary:", tokenizer.decode(summary_ids[0], skip_special_tokens=True))